# Homework 4

Gerald I. Nakata

**Github: https://github.com/ichi206/EE399/tree/main/HW4**

In [8]:
# Import Block
import numpy as np
import matplotlib.pyplot as plt
import scipy.optimize as opt
from scipy.io import loadmat

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler

# Torch imports
import torch
import torch.nn as nn
import torch.optim as optim

# PCA & MNIST imports
from sklearn.datasets import fetch_openml
from sklearn.decomposition import PCA

from torch.utils.data import DataLoader, TensorDataset

import keras
from keras.models import Sequential
from keras.layers import Dense, LSTM


## Problem 1

In [3]:
# Data
X=np.arange(0,31)
Y=np.array([30, 35, 33, 32, 34, 37, 39, 38, 36, 36, 37, 39, 42, 45, 45, 41,
40, 39, 42, 44, 47, 49, 50, 49, 46, 48, 50, 53, 55, 54, 53])

In [18]:
# [i] Three conv layer feed forward neural network
class FFNN(nn.Module):
    def __init__(self, inSize=1, hid1Size=10, hid2Size=10, outSize=1):
        super(FFNN, self).__init__()
        self.fc1 = nn.Linear(inSize, hid1Size)
        self.fc2 = nn.Linear(hid1Size, hid2Size)
        self.fc3 = nn.Linear(hid2Size, outSize)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x


In [33]:
# [ii] Use first 20 pts as training and next 10 as test

# Hyperparameters
input_size = 1
output_size = 1

# Convert arrays to tensors
Xtrain = torch.from_numpy(X[:20]).float().reshape(-1, input_size)
Ytrain = torch.from_numpy(Y[:20]).float().reshape(-1, output_size)
Xtest = torch.from_numpy(X[-10:]).float().reshape(-1, input_size)
Ytest = torch.from_numpy(Y[-10:]).float().reshape(-1, output_size)

model = FFNN(inSize=1, hid1Size=20, hid2Size=5, outSize=1)
optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.MSELoss()

# Train
num_epochs = 200
for epoch in range(num_epochs):
    outputs = model(Xtrain)
    loss = criterion(outputs, Ytrain)

    # Backpropagation
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if (epoch % 10 == 0):
        print(f"Epoch {epoch}: Loss={loss.item():.4f}")

# Test
model.eval()
with torch.no_grad():
    Ypred = model(Xtest)

test_loss = criterion(Ypred, Ytest)
print(f"Test loss: {test_loss.item():.4f}")

Epoch 0: Loss=1511.5543
Epoch 10: Loss=1003.8231
Epoch 20: Loss=675.8133
Epoch 30: Loss=456.8490
Epoch 40: Loss=310.6668
Epoch 50: Loss=213.0742
Epoch 60: Loss=147.9206
Epoch 70: Loss=104.4236
Epoch 80: Loss=75.3846
Epoch 90: Loss=55.9980
Epoch 100: Loss=43.0553
Epoch 110: Loss=34.4147
Epoch 120: Loss=28.6461
Epoch 130: Loss=24.7950
Epoch 140: Loss=22.2240
Epoch 150: Loss=20.5075
Epoch 160: Loss=19.3616
Epoch 170: Loss=18.5966
Epoch 180: Loss=18.0858
Epoch 190: Loss=17.7448
Test loss: 181.2215


In [96]:
# [iii] Use first 10 pts as training and next 10 as test

# Hyperparameters
input_size = 1
output_size = 1

# Convert arrays to tensors
Xtrain2 = torch.from_numpy(X[:10]).float().reshape(-1, input_size)
Ytrain2 = torch.from_numpy(Y[:10]).float().reshape(-1, output_size)

model2 = FFNN(inSize=1, hid1Size=20, hid2Size=5, outSize=1)
optimizer2 = optim.SGD(model2.parameters(), lr=0.01)

# Train
num_epochs2 = 30
for epoch in range(num_epochs2):
    outputs2 = model2(Xtrain2)
    loss2 = criterion(outputs2, Ytrain2)

    # Backpropagation
    optimizer2.zero_grad()
    loss2.backward()
    optimizer2.step()
    if (epoch % 10 == 0):
        print(f"Epoch {epoch}: Loss={loss2.item():.4f}")

# Test
model2.eval()
with torch.no_grad():
    Ypred = model2(Xtest)

test_loss = criterion(Ypred, Ytest)
print(f"Test loss: {test_loss.item():.4f}")

Epoch 0: Loss=1242.3337
Epoch 10: Loss=129.1992
Epoch 20: Loss=6.5838
Test loss: 245.9613


[iv] Compare the fit models in HW1 to the neural nets.

The test loss of the neural nets is worse than even the 19th degree polynomial in HW1, which paired with the comparison to the test loss demonstrates that neural nets have a proclivity to overfit to data rather quickly.

# Problem 2

In [3]:
# [i] Compute first 20 PCA modes of MNIST data
# Load the MNIST dataset
mnist = fetch_openml('mnist_784')
X, Y = mnist.data / 255., mnist.target

c:\Users\ginak\anaconda3\lib\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [102]:
# Perform PCA with 20 components
pca = PCA(n_components=20)
pca.fit(X)
#X_pca = pca.fit(X)
print("First 20 PCA Modes:")
print(pca.components_)
print("Explained Variance: " + str(pca.explained_variance_ratio_))

First 20 PCA Modes:
[[-2.34759338e-18 -6.29140696e-19  1.23041352e-19 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [-3.03168679e-17 -1.54113960e-18 -1.32578379e-18 ...  0.00000000e+00
   0.00000000e+00  0.00000000e+00]
 [ 1.85638963e-17 -1.64115091e-17  1.55878996e-18 ... -0.00000000e+00
  -0.00000000e+00 -0.00000000e+00]
 ...
 [-1.73564523e-17 -4.62190910e-17  3.58883296e-19 ... -0.00000000e+00
  -0.00000000e+00 -0.00000000e+00]
 [-8.05014655e-17 -3.36285756e-17  4.92940537e-17 ... -0.00000000e+00
  -0.00000000e+00 -0.00000000e+00]
 [-8.06369294e-17  2.27895735e-17  3.40166631e-17 ... -0.00000000e+00
  -0.00000000e+00 -0.00000000e+00]]
Explained Variance: [0.09746116 0.07155445 0.06149531 0.05403385 0.04888934 0.04305227
 0.03278262 0.02889642 0.02758364 0.0234214  0.02106688 0.02037553
 0.01707061 0.01694019 0.01583378 0.01486337 0.01319264 0.01278985
 0.01187208 0.01152689]


In [4]:
# [ii] Train a feed forward neural net on the MNIST data to classify digits. Compare results to LSTM, SVM, and decision tree classifiers.
Y = (mnist['target'].values).astype(np.int)

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert to tensors
X_train = torch.from_numpy(X_train).float()
Y_train = torch.from_numpy(Y_train).long()
X_test = torch.from_numpy(X_test).float()
Y_test = torch.from_numpy(Y_test).long()

# Data loaders
train_dataset = torch.utils.data.TensorDataset(X_train, Y_train)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataset = torch.utils.data.TensorDataset(X_test, Y_test)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=64, shuffle=False)

C:\Users\ginak\AppData\Local\Temp\ipykernel_31404\2794084576.py:2: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  Y = (mnist['target'].values).astype(np.int)


In [107]:
# Create FFNN for MNIST data
class FFNNM(nn.Module):
    def __init__(self, inSize=784, hid1Size=256, hid2Size=128, outSize=10):
        super(FFNNM, self).__init__()
        self.fc1 = nn.Linear(inSize, hid1Size)
        self.fc2 = nn.Linear(hid1Size, hid2Size)
        self.fc3 = nn.Linear(hid2Size, outSize)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.softmax(self.fc3(x))
        return x

In [115]:
# 
device = torch.device('cpu')
model = FFNNM().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01)
criterion = nn.CrossEntropyLoss()


# Train
num_epochs = 15
for epoch in range(num_epochs):
    total_loss = 0.0
    for i, (inputs, labels) in enumerate(train_loader, 0):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print("Epoch %d loss: %.3f" % (epoch + 1, total_loss / len(train_loader)))

# Test
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data
        images, lables = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == lables).sum().item()
print("Accuracy of the net on the %d test images: %.2f %%" % (len(test_loader) * 64, 100 * correct / total))

Epoch 1 loss: 2.294
Epoch 2 loss: 2.213
Epoch 3 loss: 1.975
Epoch 4 loss: 1.830
Epoch 5 loss: 1.742
Epoch 6 loss: 1.669
Epoch 7 loss: 1.630
Epoch 8 loss: 1.589
Epoch 9 loss: 1.571
Epoch 10 loss: 1.560
Epoch 11 loss: 1.553
Epoch 12 loss: 1.547
Epoch 13 loss: 1.543
Epoch 14 loss: 1.539
Epoch 15 loss: 1.535
Accuracy of the net on the 14016 test images: 93.20 %


In [9]:
# Load the MNIST dataset
from keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Preprocess the data
x_train = x_train.reshape(x_train.shape[0], 28, 28).astype('float32') / 255
x_test = x_test.reshape(x_test.shape[0], 28, 28).astype('float32') / 255

# Convert the labels to categorical one-hot encoding
num_classes = 10
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [10]:

# Define the LSTM model
model = Sequential()
model.add(LSTM(128, input_shape=(28, 28)))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(x_train, y_train, epochs=10, batch_size=128, validation_data=(x_test, y_test))

# Evaluate the model
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])


Epoch 1/10
469/469 [==============================] - 25s 47ms/step - loss: 0.5307 - accuracy: 0.8280 - val_loss: 0.1821 - val_accuracy: 0.9427
Epoch 2/10
469/469 [==============================] - 39s 82ms/step - loss: 0.1478 - accuracy: 0.9553 - val_loss: 0.1274 - val_accuracy: 0.9599
Epoch 3/10
469/469 [==============================] - 30s 65ms/step - loss: 0.1047 - accuracy: 0.9682 - val_loss: 0.0876 - val_accuracy: 0.9722
Epoch 4/10
469/469 [==============================] - 44s 93ms/step - loss: 0.0789 - accuracy: 0.9761 - val_loss: 0.0704 - val_accuracy: 0.9775
Epoch 5/10
469/469 [==============================] - 45s 95ms/step - loss: 0.0658 - accuracy: 0.9798 - val_loss: 0.0663 - val_accuracy: 0.9790
Epoch 6/10
469/469 [==============================] - 43s 92ms/step - loss: 0.0524 - accuracy: 0.9839 - val_loss: 0.0706 - val_accuracy: 0.9782
Epoch 7/10
469/469 [==============================] - 45s 95ms/step - loss: 0.0432 - accuracy: 0.9868 - val_loss: 0.0585 - val_accuracy: